In [8]:
import logging
from calendar import month

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

# Acorn Dataset
import importlib
import custom_datasets.aus_acorn_sat_data
importlib.reload(custom_datasets.aus_acorn_sat_data)

import torch
from torch.utils.data import DataLoader

print(f"Torch version {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA Available: {torch.cuda.is_available()}")

from config import ROOT_DIR
import os
model_path = os.path.join(ROOT_DIR, '.models', 'acorn_ffnn.pth')

dataset_train = custom_datasets.aus_acorn_sat_data.AusAcornSatData(mode="max")
dataset_test = custom_datasets.aus_acorn_sat_data.AusAcornSatData(mode="max", train=False)

print (dataset_train.min_values)
print (dataset_train.max_values)

original_dataset = dataset_train

train_loader = DataLoader(dataset_train, batch_size=1024, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=1024, shuffle=False)

print(f"Dataset size: {len(dataset_train)}")
print(f"Batch size: {train_loader.batch_size}")
print(f"Number of batches: {len(train_loader)}")

# Shape verification
batch_x, batch_y = next(iter(train_loader))
logger.info(f"Batch shapes: x={batch_x.shape}, y={batch_y.shape}")
logger.info(f"Batch dtypes: x={batch_x.dtype}, y={batch_y.dtype}")


Torch version 2.7.1+cu118
CUDA Available: True


INFO:__main__:Batch shapes: x=torch.Size([1024, 5]), y=torch.Size([1024, 1])
INFO:__main__:Batch dtypes: x=torch.float32, y=torch.float32


{'max': np.float64(-3.3), 'lat': np.float64(-43.49), 'lon': np.float64(113.67), 'elevation': np.int64(2), 'days_since_start': np.int64(0), 'days_since_start_of_year': np.int32(1)}
{'max': np.float64(49.6), 'lat': np.float64(-10.58), 'lon': np.float64(153.47), 'elevation': np.int64(1482), 'days_since_start': np.int64(42003), 'days_since_start_of_year': np.int32(366)}
Dataset size: 319620
Batch size: 1024
Number of batches: 313


In [4]:
import torch.nn as nn

class FFNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)

In [26]:
import torch.optim as optim
from config import ROOT_DIR
import os
num_epochs = 10

model = FFNN().to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

best_loss = float('inf')

logger.info("Starting Training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        batch_x = x.to(device)
        batch_y = y.to(device)
        optimizer.zero_grad()

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        total_loss = 0
        for x, y in test_loader:
            batch_x = x.to(device)
            batch_y = y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
        avg_loss = total_loss / len(test_loader)
        logger.info(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {avg_loss:.8f}")

        logger.info(f"Best Loss: {best_loss:.8f}, Current Loss: {avg_loss:.8f}")
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), model_path)
        else:
            logger.info("Loss did not improve, stopping training.")
            break

model.eval()

INFO:__main__:Starting Training...
INFO:__main__:Epoch 1/10, Test Loss: 0.0046
INFO:__main__:Best Loss: inf, Current Loss: 0.0046
INFO:__main__:Epoch 2/10, Test Loss: 0.0045
INFO:__main__:Best Loss: 0.0046, Current Loss: 0.0045
INFO:__main__:Epoch 3/10, Test Loss: 0.0045
INFO:__main__:Best Loss: 0.0045, Current Loss: 0.0045
INFO:__main__:Epoch 4/10, Test Loss: 0.0044
INFO:__main__:Best Loss: 0.0045, Current Loss: 0.0044
INFO:__main__:Epoch 5/10, Test Loss: 0.0044
INFO:__main__:Best Loss: 0.0044, Current Loss: 0.0044
INFO:__main__:Epoch 6/10, Test Loss: 0.0044
INFO:__main__:Best Loss: 0.0044, Current Loss: 0.0044
INFO:__main__:Epoch 7/10, Test Loss: 0.0044
INFO:__main__:Best Loss: 0.0044, Current Loss: 0.0044
INFO:__main__:Loss did not improve, stopping training.


FFNN(
  (net): Sequential(
    (0): Linear(in_features=5, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [10]:
from datetime import date, timedelta

model = FFNN().to(device)
model.load_state_dict(torch.load(model_path, weights_only=True))

dataset_start = date(1910, 1, 1)
start = date.today()
for i in range(12):
    prediction_date = start + timedelta(days=i*30)
    days_between = (prediction_date - dataset_start).days

    test_data =  {
        'lat': -33.9607047,
        'lon': 151.128669,
        'elevation': 29.0,
        'days_since_start': days_between,
        'days_since_start_of_year': (prediction_date - date(prediction_date.year, 1, 1)).days
    }

    test_tensor = original_dataset.normalize(test_data).to(device)

    with torch.no_grad():
        output = model(test_tensor)
    value = output.cpu().item()

    predicted_temperature = original_dataset.denormalize(value)
    print(f"PMax {prediction_date}, lat {test_data['lat']}, lon {test_data['lon']}, elevation {test_data['elevation']}: {predicted_temperature:.2f}°C")

PMax 2026-02-15, lat -33.9607047, lon 151.128669, elevation 29.0: 27.22°C
PMax 2026-03-17, lat -33.9607047, lon 151.128669, elevation 29.0: 26.41°C
PMax 2026-04-16, lat -33.9607047, lon 151.128669, elevation 29.0: 24.34°C
PMax 2026-05-16, lat -33.9607047, lon 151.128669, elevation 29.0: 20.43°C
PMax 2026-06-15, lat -33.9607047, lon 151.128669, elevation 29.0: 18.26°C
PMax 2026-07-15, lat -33.9607047, lon 151.128669, elevation 29.0: 17.55°C
PMax 2026-08-14, lat -33.9607047, lon 151.128669, elevation 29.0: 19.04°C
PMax 2026-09-13, lat -33.9607047, lon 151.128669, elevation 29.0: 22.00°C
PMax 2026-10-13, lat -33.9607047, lon 151.128669, elevation 29.0: 23.72°C
PMax 2026-11-12, lat -33.9607047, lon 151.128669, elevation 29.0: 24.93°C
PMax 2026-12-12, lat -33.9607047, lon 151.128669, elevation 29.0: 26.25°C
PMax 2027-01-11, lat -33.9607047, lon 151.128669, elevation 29.0: 28.18°C
